# Chapter 8 -  Detecting Overlapping communities

The problem of graph clustering is well studied, in particular the case where the vertices are partitioned into
non-overlapping communities.

Here, we look at the problem of graph clustering where:
* vertices can be part of several communities (overlapping communities)
* vertices can be part of no community ("noise" vertices)

We explore the following three methods:
* methods based on finding overlapping cliques (a clique is a complete subgraph)
* methods based on splitting vertices into multiple personae, and
* methods based on clustering the edges.

We also look at some post-processing based on Community Association Strength scores (CAS), which can be used after running the above, or some graph partioning algorithm (such as ECG ou Leiden)

We illustrate those methods using the small Karate Club graph.
Next we conaider larger graphs: a word association graph and artificial ABCD benchmark graphs. 


## New requirements

* overlapping NMI measure (oNMI): download and compile from: https://github.com/aaronmcdaid/Overlapping-NMI 


In [ ]:
using PythonCall
using CondaPkg
using Random
using DelimitedFiles
using Combinatorics
using StatsBase

In [ ]:
ig = pyimport("igraph")
random = pyimport("random")
pd = pyimport("pandas")
pickle = pyimport("pickle")
plt = pyimport("matplotlib.pyplot")
pyimport("partition_igraph")

In [ ]:
datadir = "../Datasets/"
## oNMI executable:
oNMI = "../oNMI/onmi"             ## overlapping NMI executable

In [ ]:
## calls the oNMI executable, format of inputs: list of lists (communities)
function compute_oNMI(First::Vector{Vector{Int}},
    Second::Vector{Vector{Int}},
    oNMI_path::String)
    fn1 = "__" * string(rand(UInt))[1:10]
    fn2 = "__" * string(rand(UInt))[1:10]

    open(fn1, "w") do f
        for row in First
            println(f, join(row, " "))
        end
    end

    open(fn2, "w") do f
        for row in Second
            println(f, join(row, " "))
        end
    end
    output = read(`$oNMI_path $fn1 $fn2`, String)
    tokens = split(output)
    x = parse(Float64, tokens[2])
    rm(fn1)
    rm(fn2)
    return x
end


In [ ]:
## assign colors and shapes w.r.t. overlapping clusters
## white: no cluster
## black square: overlap
## the rest are shown as colored circles
function color_nodes(g, communities, greyscale=false)
    g.vs["_oc"] = [pylist([]) for i in 0:pyconvert(Int, g.vcount())-1]
    for i in 1:length(communities)
        for j in communities[i]
            g.vs.find(j)["_oc"].append(i)
        end
    end
    if greyscale
        pal = ig.drawing.colors.GradientPalette("white", "black", n=length(communities) + 2)
    else
        pal = ig.drawing.colors.ClusterColoringPalette(n=length(communities))
    end
    g.vs["shape"] = "circle"
    for v in g.vs
        if length(v["_oc"]) == 0
            v["color"] = "white"
        else
            if length(v["_oc"]) > 1
                v["color"] = "black"
                v["shape"] = "square"
            else
                if greyscale
                    v["color"] = pal[v["_oc"][0]]
                else
                    v["color"] = pal[v["_oc"][0]-1]
                end
            end
        end
    end
end

# 1. CPM (clique percolation method)

The first algorithm we consider is the Clique Percolation Method, which can
be summarized as:
* fix the clique size $k$ (typically $k$=3 or 4)
* for each $k$-clique, join all other $k$-cliques with $k-1$ vertices in common, in turn (the percolation)
* continue until all $k$-cliques are exhausted

Is is based on:

Derényi I., *et al.*, Clique percolation in random networks, Phys. Rev. Lett., 2005, vol. 94 (pg. 160-202)


In [ ]:
function CPM(g, k=3)
    cls = collect(map(Set, g.cliques(min=k, max=k)))
    edgelist = []
    for i in 0:length(cls)-1
        push!(edgelist, (i, i))
    end
    for (i, j) in combinations(0:length(cls)-1, 2)
        if length(intersect(cls[i+1], cls[j+1])) >= (k - 1)
            push!(edgelist, (i, j))
        end
    end
    cg = ig.Graph(edgelist, directed=false)
    clusters = cg.connected_components()
    L = []
    members = Set()
    for cluster in clusters
        members = Set()
        for i in cluster
            push!(members, cls[pyconvert(Int, i)+1]...)
        end
        push!(L, Set(g.vs[pyset(members)]["name"]))
    end
    return L
end

#### CPM on the Zachary Graph

We illustrate the different CPM-based algorithms with the well-known Karate Club dataset, which model interaction between 34 members. The 2 communities correspond to groups forming after a split in two "factions". Modularity-based algorithms usually find 4 or 5 communities.
Below, we color the nodes according to the 2 factions after the split.  

In [ ]:
## Zachary graph and its two communities
zac = ig.Graph.Read_Ncol(datadir * "Zachary/zachary.edgelist", directed=false)
c = vec(readdlm(datadir * "Zachary/zachary.communities", Int))
zac.vs["comm"] = [c[parse(Int, pyconvert(String, x["name"]))+1] for x in zac.vs]

## plotting parameters
zac.vs["size"] = 12
zac.es["color"] = "gainsboro"
pal = ig.drawing.colors.ClusterColoringPalette(n=maximum(zac.vs["comm"]) + 1)
zac.vs["color"] = [pal[i] for i in zac.vs["comm"]]

# ## plot
ig.plot(zac, bbox=(0, 0, 300, 300))

####  Running the CPM algorithm

* We run the CPM algorithm as is on the Karate graph.
* Nodes that belong to 2 or more clusters are represented as squares.
* You can select col="grey" for greyscale, but this is hard to distinguish with several clusters
* We obtain one large community, two small ones and two orphan nodes (shown in white)

In [ ]:
## ground-truth communities
zac_gt = []
for i in Set(pyconvert(Vector{Int}, zac.vs["comm"]))
    push!(zac_gt, [v["name"] for v in zac.vs if pyconvert(Int, v["comm"]) == i])
end

In [ ]:
## CPM with k=3
X = CPM(zac, 3)
color_nodes(zac, X, false)
println("oNMI:", compute_oNMI([collect(i) for i in X], zac_gt))

## plot
ig.plot(zac, bbox=(0, 0, 300, 300))

In [ ]:
## CPM with k=4
X = CPM(zac, 4)
color_nodes(zac, X, false)
print("oNMI:", compute_oNMI([collect(i) for i in X], zac_gt))

## plot
ig.plot(zac, bbox=(0, 0, 300, 300))

In [ ]:
## filter edges with small ECG weight (threshold or under)
threshold = 0 ## filer edges with NO vote
random.seed(123)
zac.es["ecg_w"] = zac.community_ecg(ens_size=32, min_weight=0).W
zac_sg = zac.subgraph_edges([e for e in zac.es if pyconvert(Float64, e["ecg_w"]) > threshold])
X = CPM(zac_sg, 3)
color_nodes(zac, X, false)
print("oNMI:", compute_oNMI([collect(i) for i in X], zac_gt))

## plot
ig.plot(zac, bbox=(0, 0, 300, 300))

In [ ]:
# random clusterings with same sizes as above
random.seed(123)
Nodes = []
for x in X
    Nodes=[Nodes; collect(x)]
end
Sizes = [[0]; cumsum(length.(X))]
Results = []
for rep in 1:100
    R = []
    P = shuffle(Nodes)
    for s in 1:length(Sizes)-1:
        push!(R,(P[Sizes[s]:Sizes[s+1]]))
    end
    push!(Results,(compute_oNMI(R,zac_gt)))
end
## report mean and stdv
println("mean:", mean(Results), "stdv:", std(Results))

# 2. Ego-Splitting method

The Ego-Splitting framework is based on paper by A. Epasto, S. Lattanzi and R.P. Leme at KDD 2017:

https://www.kdd.org/kdd2017/papers/view/ego-splitting-framework-from-non-overlapping-to-overlapping-clusters


In summary, the steps are:
* For each vertex $v$:
 * build the ego-net for $v$ (minus self)
 * cluster this ego-net using a local method, such as label propagation (LP) or connected components (CC)
 * "split" vertex $v$ into one persona per ego-net cluster
* Cluster this new graph (with duplicated vertices) with some graph partitioning algorithm such as LP or ECG. 
 * We can set a minimum community size to avoid tiny ones.

The original paper uses a LP method based on the Potts model, but we will use the Label Propagation from Raghavan *et. al.* which is implemented in igraph.


In [ ]:
function EgoSplit(G, split="CC", algo="LP")
    g = G.copy()
    ## implement ego-split approach with LP+LP and LP+ECG
    g.vs["original"] = g.vs["name"]
    ## use the vertex names to avoid issues when vertices are re-mapped ...
    names = g.vs["name"]
    ## step 1 - ego-net splits
    for nm in names
        v = g.vs.find(nm).index
        n = g.neighbors(v)
        sg = g.subgraph(n)
        if split == "LP"
            x = sg.community_label_propagation().membership
        else
            x = sg.connected_components().membership
        end
        if minimum(pyconvert(Vector{Int}, x)) == -1
            x = [i + 1 for i in x]
        end
        for j in Set(x)
            g.add_vertex(name=nm + Py(".") + pystr(j), original=nm)
        end
        l = sg.vs["name"]
        for j in 0:length(x)-1
            g.add_edge(nm + Py(".") + pystr(x[j]), l[j])
        end
        g.delete_vertices(v)
    end
    ## step 2 -- cluster w.r.t. multiple personae
    if algo == "LP"
        cl = g.community_label_propagation()
    else
        cl = g.community_ecg(ens_size=32)
    end
    C = [Set(sg.vs["original"]) for sg in cl.subgraphs()]
    return C
end

In [ ]:
## ego-split
random.seed(1)
X = EgoSplit(zac, "LP") ## pick final algorithm as parameter (LP or ECG)
X = [Set(l) for l in X if length(l) >= 3] ## min community size set to 3
color_nodes(zac, X, false)
println("oNMI:", compute_oNMI([collect(i) for i in X], zac_gt))
ig.plot(zac, bbox=(0, 0, 300, 300))

# 3. Edge Clustering 

We can obtain overlapping communities by clustering edges instead of vertices.
The algorithm can be described as follows:
* for each pair of edges sharing a node, say $(i,k)$ and $(j,k)$, compute some similarity measure between the neighborhoods of vertices $i$ and $j$, such as the Jaccard measure
* perform hierarchical clustering on the edges with this similarity matrix

It is based on:

Ahn, YY., Bagrow, J., Lehmann, S. Link communities reveal multiscale complexity in networks. Nature 466, 761–764 (2010). https://doi.org/10.1038/nature09182

This can be implemented by considering the connected components for the line-graph of the original graph using varying thresholds for the Jaccard measure.
* line graph Lg(G) represents ties between edges of G
* Lg(G) nodes are edges in G
* edges sharing a node in G are linked by an edge in Lg(G)

We pick the "best" clustering in the hierarchy based on the modularity scores on the line graph.


In [ ]:
function Jaccard(a, b)
    x = length(intersect(Set(a), Set(b))) / length(union(Set(a), Set(b)))
    return x
end

function weightedLinegraph(g)
    lg = g.linegraph()
    w = []
    for e in lg.es
        A = Set(g.es[e.tuple[0]].tuple)
        B = Set(g.es[e.tuple[1]].tuple)
        x = collect(union(setdiff(A, B), setdiff(B, A)))
        push!(w, Jaccard(g.neighbors(x[1]), g.neighbors(x[2])))
    end
    lg.es["weight"] = w
    return lg
end

function edgeCluster(g)
    q = -999
    D = weightedLinegraph(g)
    for th in sort(pyconvert(Vector{Float64}, collect(Set(D.es["weight"]))))
        ## filter edges w.r.t. similarity and find CC
        dg = D.copy()
        dg.delete_edges(pylist([e for e in dg.es if pyconvert(Float64, e["weight"]) <= th]))
        cc = dg.connected_components().membership
        mod = pyconvert(Float64, D.modularity(cc))
        if mod > q
            q = mod
            g.es["lc"] = cc
        end
    end
    ## Now gather the nodes for each edge cluster
    L = []
    for i in 0:pyconvert(Int, maximum(g.es["lc"]))
        sg = g.subgraph_edges(pylist([e for e in g.es if pyconvert(Int, e["lc"]) == i]))
        push!(L, sg.vs["name"])
    end
    return L
end

In [ ]:
## cluster
X = edgeCluster(zac) ## pick final algorithm as parameter (LP or ECG)
X = [Set(l) for l in X if length(l) >= 3] ## min community size set to 3
# print("oNMI:",compute_oNMI([collect(i) for i in X], zac_gt))
color_nodes(zac, X, false)
ig.plot(zac, bbox=(0, 0, 300, 300))

### Post-processing with CAS scores

From the previous result, drop community memberships with low CAS scores.


In [ ]:
## community association strength for partitions
function cas(G, A)
    deg = pyconvert(Vector{Int}, G.degree())
    deg_int = [sum([A[pyconvert(Int, i)+1] == A[pyconvert(Int, j)+1] for i in G.neighbors(j)]) for j in 0:pyconvert(Int, G.vcount())-1]
    Vol = sum(deg)
    Vol_A = zeros(Int, pyconvert(Int, maximum(A)) + 1)
    for i in 1:pyconvert(Int, G.vcount())
        Vol_A[A[i]+1] += deg[i]
    end
    return deg_int ./ deg - ([Vol_A[A[i]+1] for i in 1:pyconvert(Int, G.vcount())] - deg) ./ Vol
end

In [ ]:
## drop comminity memberships with low scores
threshold = 0.1
Y = []
for i in 1:length(X)
    zac.vs["_com"] = pylist(1:pyconvert(Int, zac.vcount())) ## initialize each node in its own community
    for x in X[i]
        zac.vs.find(x)["_com"] = 0 ## consider community X[i] only
    end
    c = cas(zac, pyconvert(Vector{Int}, zac.vs["_com"])) ## cas w.r.t. X[i] for nodes in that community only
    push!(Y, Set([zac.vs[i]["name"] for i in 0:length(c)-1 if c[i+1] >= threshold])) ## keep only cas results above threshold
end
## plot and compute oNMI
color_nodes(zac, Y, false)
## plot
print("oNMI:", compute_oNMI([collect(i) for i in Y], zac_gt))
ig.plot(zac, bbox=(0, 0, 300, 300))

# Word Association Graph Example

We consider a graph built from the Word Association dataset (U of South Florida) based on:

* G. Palla *et al.*, "Uncovering the overlapping structure of complex networks in nature and society", Nature 435, 814-818 (2005).

In a nutshell, we build a graph with edges between pairs of similar words. We used a threshold of $w^*=.025$ for the association strength and use $k=4$ for the clique size. We use this dataset to illustrate the usefulness of overlapping clusters to discover various contexts of words. 

We look at two versions of CPM:
* using the ECG-based version, and
* using the association strength as edge weight.


In [ ]:
## build the graph
wg = ig.Graph.Read_Ncol(datadir * "Words/words.txt", names=true, directed=false, weights=true)
wg = wg.simplify(combine_edges="sum") ## sum association strength scores
wg = wg.subgraph_edges([e for e in wg.es if pyconvert(Float64, e["weight"]) >= 0.025]) ## prune low weight edges
wg.vs["label"] = wg.vs["name"]
println(wg.vcount(), " nodes and ", wg.ecount(), " edges")

In [ ]:
## use ECG-based weights
random.seed(123)

for word in ["MATH", "DOG", "MONEY"] ## you can try other words; all words are CAPITALIZED

    ## get 2-hop ego-net
    v = wg.vs.find(name=word)
    n = wg.neighborhood(v, order=2)
    sg = wg.subgraph(n)

    ## filter edges w.r.t. ECG score
    threshold = 0
    random.seed(123)
    sg.es["ecg_w"] = sg.community_ecg(ens_size=32, min_weight=0).W
    sg = sg.subgraph_edges([e for e in sg.es if pyconvert(Float64, e["ecg_w"]) > threshold])

    ## cluster and show results containing the given word
    X = CPM(sg, 4)
    println("\nShowing clusters for the word ", word)
    for x in X
        if word in pyconvert(Set{String}, x)
            println(join(sort(collect(x)), " "))
        end
    end
end

In [ ]:
## use association scores as weights

for word in ["MATH", "DOG", "MONEY"] ## you can try other words; all words are CAPITALIZED

    ## get 2-hop ego-net
    v = wg.vs.find(name=word)
    n = wg.neighborhood(v, order=2)
    sg = wg.subgraph(n)

    ## filter edges w.r.t. association strength scores
    threshold = 0.025
    sg = sg.subgraph_edges([e for e in sg.es if pyconvert(Float64, e["weight"]) > threshold])

    ## cluster and show results containing the given word
    X = CPM(sg, 4)
    println("\nShowing clusters for the word", word)
    for x in X
        if word in pyconvert(Set{String}, x)
            println(join(sort(collect(x)), " "))
        end
    end
end

# Study over ABCD-$o^2$ benchmark graphs

We generated 1,320 ABCD-$o^2$ graphs with the following parameters:
* $n = 1,000$ nodes, no outliers
* power law degree exponent $\tau_1=2.5$ with degrees in range [5,50]
* community size degree exponent $\tau_2=1.5$ with sizes in range [50,200]
* noise parameter $0.1 \le \xi \le 0.65$
* overlap parameter $1.0 \le \eta \le 2.0$
* $d=2$ (dimension of the spatial model for overlaps) and $\rho=0$ (correlation between degree and number of community memberships)

For the CAS-based post-processing, we used thresholds $t_1 = t_2 = 0.1$ (respectively to remove or add nodes to communities).

The configuration file to build the graphs can be found in the ```Datasets/ABCDoo``` subdirectory, as well as a ```pickle``` file that contains the results from the experiments performed in `../Python_Notebooks_2nd/Chapter_8.ipynb` notebook.


In [ ]:
## load the results from the experiment above
open(datadir * "ABCDoo/abcdoo.pkl", "r") do fp
    global df = pickle.load(fp)
end
df = DataFrame(PyTable(df))

In [ ]:
## fix xi
xi = 0.1
cls = ["black", "black", "black"]
algos = ["ego-split", "ego-split+cas", "ecg+cas"]
stdv = ["ego-split(sd)", "ego-split+cas(sd)", "ecg+cas(sd)"]
style = [":", "--", "-"]
_df = df[(df.xi.==xi), :]
for j in 1:3
    s = algos[j]
    e = stdv[j]
    plt.plot(_df.eta, _df[!, s], label=s, color=cls[j], linestyle=style[j])
    plt.fill_between(_df.eta, _df[!, s] + 2 * _df[!, e], _df[!, s] - 2 * _df[!, e], alpha=0.1, color=cls[j])
end
plt.legend()
plt.grid()
plt.title(raw"ABCD-oo graphs with $\xi$=" * string(xi), fontsize=16)
plt.ylabel("oNMI", fontsize=14)
plt.xlabel(raw"$\eta$", fontsize=14)
plt.gcf()

In [ ]:
## fix eta
eta = 1.5
cls = ["black", "black", "black"]
algos = ["ego-split", "ego-split+cas", "ecg+cas"]
stdv = ["ego-split(sd)", "ego-split+cas(sd)", "ecg+cas(sd)"]
style = [":", "--", "-"]
_df = df[(df.eta.==eta), :]
for j in 1:3
    s = algos[j]
    e = stdv[j]
    plt.plot(_df.xi, _df[!, s], label=s, color=cls[j], linestyle=style[j])
    plt.fill_between(_df.xi, _df[!, s] + 2 * _df[!, e], _df[!, s] - 2 * _df[!, e], alpha=0.1, color=cls[j])
end
plt.legend()
plt.grid()
plt.title(raw"ABCD-oo graphs with $\eta$=" * string(eta), fontsize=16)
plt.ylabel("oNMI", fontsize=14)
plt.xlabel(raw"$\xi$", fontsize=14)
plt.gcf()

In [ ]:
## fix eta
eta = 1.1
#cls = ["blue","green","red","purple"]
cls = ["black", "black", "black"]
algos = ["ego-split", "ego-split+cas", "ecg+cas"]
stdv = ["ego-split(sd)", "ego-split+cas(sd)", "ecg+cas(sd)"]
style = [":", "--", "-"]
_df = df[(df.eta.==eta), :]
for j in 1:3
    s = algos[j]
    e = stdv[j]
    plt.plot(_df.xi, _df[!, s], label=s, color=cls[j], linestyle=style[j])
    plt.fill_between(_df.xi, _df[!, s] + 2 * _df[!, e], _df[!, s] - 2 * _df[!, e], alpha=0.1, color=cls[j])
end
plt.legend()
plt.grid()
plt.title(raw"ABCD-oo graphs with $\eta$=" * string(eta), fontsize=16)
plt.ylabel("oNMI", fontsize=14)
plt.xlabel(raw"$\xi$", fontsize=14)
plt.gcf()